# Evaluate evoked LFP response by optogenetic stimulations

### Load LFP and packages

Load packages

In [ ]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import os
from IPython.display import display
from ipyfilechooser import FileChooser
from scipy.stats import zscore
import json
import matplotlib.cm as cm
%matplotlib widget


In [ ]:
Channels = '//10.69.168.1/crnldata/forgetting/Aurelie/allLFPChannels_perMice.xlsx'
allchannels = pd.read_excel(Channels, index_col=0)

Choose OpenEphys folder

In [ ]:
try: # tries to retrieve dpath either from a previous run or from a previous notebook
    %store -r dpath
    #dpath=Path(dpath).parent
except:
    print("the path was not defined in store")
    #dpath = "/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/ExampleRedLines/2022_08_06/13_30_01/My_V4_Miniscope/"
    dpath = "//10.69.168.1/crnldata/forgetting/Aurelie/"


fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Go inside the folder containing the LFP raw file</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

Load LFPs data, TTL, timestamps

In [ ]:
folder_base = Path(dpath)

LFPfile = Path(f'{folder_base}\continuousDS.npy') #RawDataChannelExtractedDS.npy
LFPfile2= Path(f'{folder_base}\continuous.dat')

# Load LFPs data 
if LFPfile.exists(): # prefer loading downsample file other original file
    print('continuousDS.npy file')
    All = np.load(LFPfile, mmap_mode= 'r')
    samplerate=1000 
    numchannel=All.shape[1]
    # Load LFPs timestamps 
    for file_pathTS in folder_base.parent.parent.glob('**/continuous/*/timeStampsDS.npy'):
        print('LFPs timestamps file = ', file_pathTS)
        LFPtimestamps = np.load(file_pathTS)  
elif LFPfile2.exists():
    print('continuous.dat file')
    LFPfile = LFPfile2
    DataRec = np.fromfile(LFPfile, dtype="int16")
    filepath_metadata = Path(os.path.join(folder_base.parent.parent, f'structure.oebin'))
    if filepath_metadata.exists():    
        with open(filepath_metadata) as f:
            metadata = json.load(f)
        samplerate=metadata['continuous'][0]['sample_rate']  
        numchannel=metadata['continuous'][0]['num_channels']  
    else:
        samplerate=1000 
        numchannel=32
    All = DataRec.reshape(-1,numchannel)
    # Load LFPs timestamps 
    for file_pathTS in folder_base.parent.parent.glob('**/continuous/*/timeStamps.npy'):
        print('LFPs timestamps file = ', file_pathTS)
        LFPtimestamps = np.load(file_pathTS)  
else: 
    print('no LFPs file found')

print(numchannel, 'channels recorded')
print('sample rate =', samplerate, 'Hz')
print(round(All.shape[0]/samplerate/60), 'min of recording')

In [ ]:
nb_decimal=3 # 4 = 0.1ms precision / 3 = 1ms precision

# Load TTLs
TTL_Opto_duration=[]
for file_pathTTL in folder_base.parent.parent.glob('**/TTL/timeStamps.npy'):
    print('TTL opto file = ', file_pathTTL)
    TTL_Opto_o = np.load(file_pathTTL)
    TTL_Opto_duration =[round(TTL_Opto_o[i+1] - TTL_Opto_o[i],nb_decimal) for i in range(len(TTL_Opto_o) - 1)[::2]]
    TTL_Opto= TTL_Opto_o[::2] # remove the TTL for laser OFF, only keep TTL for laser ON. CAUTION /!/ works only if it started with a TTL for laser ON
    print(TTL_Opto.shape[0], 'opto stimulations')

Downsample LFP data to 1kHz if needed & save it

In [ ]:
if samplerate > 1000:
    datalen = np.shape(All)[0]
    new_sampling_rate = 1000 # Hz
    Nmber_points = int(datalen * new_sampling_rate / samplerate)
    AllDS=signal.resample(All, Nmber_points, axis = 0)
    LFPtimestampsDS=LFPtimestamps[::10][:-1]
    samplerate=new_sampling_rate
    np.save(f'{LFPfile.parent}/continuousDS.npy', AllDS)
    np.save(f'{file_pathTS.parent}/timeStampsDS.npy', LFPtimestampsDS)
    All=AllDS
    LFPtimestamps=LFPtimestampsDS
# eventually delete original files to gain space

Load LFPs channels

In [ ]:
mouse=[]
for mouse_name in allchannels.index:
    if mouse_name in LFPfile.__str__():
        mouse.append(mouse_name)

sCh=0 
all_ch_selected=1
if len(mouse)>1: # found multiple mouse name in the path
    if numchannel==32 : #  Only one mouse recorded
        ID = 0 # change the index if this is not the good mouse name
        print(f"/!\ Mutliple mouse names in the path = {mouse}. The n°{ID+1} was choosen automatically = {mouse[ID]}.")
        mouse=mouse[ID] 
    elif numchannel % 32 == 0: # Multiple mice recorded at the same time
        ID = 0 # choose 0 to see the first mouse recorded, 1 the second, 2 the third, 3 the fourth (only 4 mice can be recorded at the same time)
        sCh= 32*ID 
        print(f"/!\ Mutliple mice recorded at the same time = {mouse}. The n°{ID+1} was choosen automatically = {mouse[ID]}.")
        mouse=mouse[ID] 
    else: # the number of channels recorded is not a multiple of 32 = you probably selected manually which channel to record 
        ID = 0 # choose 0 to see the first mouse recorded, 1 the second, 2 the third, 3 the fourth (only 4 mice can be recorded at the same time)
        print(f"/!\ Mutliple mice recorded at the same time = {mouse}. The n°{ID+1} was choosen automatically = {mouse[ID]}.\n/!\ Channels were manually selected for the recording.")
        mouse=mouse[ID] 
        all_ch_selected=0 # not all channels were recorded
elif len(mouse)==1: # found one mouse name in the path
    mouse=mouse[0]

if mouse:
    RecordedArea=[]
    ChoosenChannels=[]
    combined=[]
    num_ch=0
    for idx, channels in enumerate(allchannels.loc[mouse]):
        region=allchannels.loc[mouse].index[idx]
        if region != 'EMG':
            if isinstance(allchannels.loc[mouse].loc[region], str): # at least 2 channels
                ch_list = channels.split('_')
                for idx, ch in enumerate(ch_list): # only take the 2 first channels (change the order in the excel file if you want another pair)
                    LFP=All[:,int(ch)+sCh] if all_ch_selected else All[:,num_ch]
                    num_ch+=1
                    locals()[f'{region}_{idx}']=LFP
                    RecordedArea.append(f'{region}_{idx}')    
                    combined=zscore(LFP[:,np.newaxis]) if len(combined)==0 else np.append(combined, zscore(LFP[:,np.newaxis]), axis=1)
                    ChoosenChannels.append(ch)  if all_ch_selected else ChoosenChannels.append([num_ch])    
            elif isinstance(allchannels.loc[mouse].loc[region], (np.int64, int)): # only one channel
                ch=allchannels.loc[mouse].loc[region]  
                idx=0
                ChoosenChannels.append(str(ch)) if all_ch_selected else ChoosenChannels.append(num_ch)
                LFP=All[:,ch+sCh] if all_ch_selected else All[:,num_ch]
                num_ch+=1
                locals()[f'{region}_{idx}']=LFP
                RecordedArea.append(f'{region}_{idx}')    
                combined=zscore(LFP[:,np.newaxis]) if len(combined)==0 else np.append(combined, zscore(LFP[:,np.newaxis]), axis=1)
    print(mouse)
    print(RecordedArea)
    print(ChoosenChannels) 
else:
    print("/!\ No mouse name found in the path OR in the excel file 'allLFPChannels_perMice.xlsx'")
    mouse='' # fill mouse name
    RecordedArea=['PFC_0','PFC_1'] # change 
    PFC_0=All[:,0] # change 
    PFC_1=All[:,0] # change 
    combined=np.stack([zscore(PFC_0), zscore(PFC_1)], axis=1) # change 

Check TTL durations

In [ ]:
plt.close('all')
plt.figure(figsize=(6, 3))
plt.stairs(np.array(TTL_Opto_duration)*1000)
plt.xlabel('# Opto stimulations')
plt.ylabel('Opto stimulation durations (ms)')
plt.tight_layout()
plt.show()

### Get response of each brain region

In [ ]:
AllEPSPs=[]
for ttl in TTL_Opto:   
    idx = (np.abs(LFPtimestamps - ttl)).argmin() # find the closest LFP timestamps to the TTL
    AllEPSPs.append(combined[idx-round(0.5*samplerate):idx+round(0.5*samplerate), :]) #500 ms before and after TTL

plt.close()
plt.figure(figsize=(6, 3))
time_axis = np.linspace(-500, 500, np.shape(AllEPSPs)[1]) 
mEPSPs=np.mean(AllEPSPs, axis=0)
colors = cm.rainbow(np.linspace(0, 1, np.shape(mEPSPs)[1]))
for i in np.arange(0,np.shape(AllEPSPs)[2]):
    plt.plot(time_axis, mEPSPs[:,i], label=f'{RecordedArea[i]}', color=colors[i])
plt.xlabel("Time (ms)")
plt.ylabel("Averaged EPSPs")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncols=1)
plt.subplots_adjust(right=.5)
plt.xlim([-25, 50])
plt.title(f'Mouse = {mouse}, Stim opto duration = {np.unique(TTL_Opto_duration)[0]*1000} - {np.round(np.unique(TTL_Opto_duration)[-1]*1000,2)} ms \nfilename = {folder_base.parts[-6]}', fontsize=12) # Change to indicate where is located that LFP
plt.tight_layout()
plt.show()

### Get response for one brain region

Choose brain region

In [ ]:
Selected_region='oPFC_0' # to change
SelectedLFP=locals()[Selected_region]

In [ ]:
nb_differentStim=len(np.unique(TTL_Opto_duration)) # how many different opto stim are performed in the arduino protocol (here a loop of 19 stim with different durations)
DurStim=np.unique(TTL_Opto_duration)*1000 # durations of each unique stim (from 1 ms to 20 ms with a step of 1)

halfdurEPSP=0.1 #sec
analyseWindow= 0.025 #sec post stim

AllStim={}
for i, ttl  in enumerate(TTL_Opto):
    idx = (np.abs(LFPtimestamps - ttl)).argmin() # find the closest LFP timestamps to the TTL    
    if str(TTL_Opto_duration[i]*1000) in AllStim:
        AllStim[str(TTL_Opto_duration[i]*1000) ]=pd.concat([AllStim[str(TTL_Opto_duration[i]*1000) ],pd.DataFrame(SelectedLFP[idx-round(halfdurEPSP*samplerate):idx+round(halfdurEPSP*samplerate)])],axis=1)
    else:
        AllStim[str(TTL_Opto_duration[i]*1000) ]=pd.DataFrame(SelectedLFP[idx-round(halfdurEPSP*samplerate):idx+round(halfdurEPSP*samplerate)])
   
AllStim_sorted = dict(sorted(AllStim.items()))
meanAllStim= pd.DataFrame([np.mean(AllStim_sorted[key], axis=1) for key in AllStim_sorted.keys()]).T
meanAllStim= pd.DataFrame([meanAllStim[key]-np.mean(meanAllStim[key][int((halfdurEPSP*samplerate)-(halfdurEPSP*samplerate)):int(halfdurEPSP*samplerate)]) for key in meanAllStim.columns]).T
meanAllStim.columns=DurStim #convert columns names in ms

"""                         
Amplitude=[]
Max=[]
HalfMaxWidth=[]
for i in meanAllStim.columns:
    y=meanAllStim[i][round(halfdurEPSP*samplerate):round((halfdurEPSP+analyseWindow)*samplerate)]
    peaks, properties = signal.find_peaks(y, height=min(y), prominence=0)
    if len(peaks) > 0:
        widths, width_heights, left_ips, right_ips = signal.peak_widths(y, peaks, rel_height=0.5)
        biggest_peak_idx = np.argmax(properties["peak_heights"])
        biggest_peak = peaks[biggest_peak_idx]
        half_width = widths[biggest_peak_idx] / 2
        Amplitude.append(properties["prominences"][biggest_peak_idx])
        Max.append(properties["peak_heights"][biggest_peak_idx])
        HalfMaxWidth.append(half_width)
    else:
        Amplitude.append(np.nan)
        Max.append(np.nan)
        HalfMaxWidth.append(np.nan)
"""
Amplitude=meanAllStim[round(halfdurEPSP*samplerate):round((halfdurEPSP+analyseWindow)*samplerate)].max()-meanAllStim[round(halfdurEPSP*samplerate):round((halfdurEPSP+analyseWindow)*samplerate)].min()
Max=meanAllStim[round(halfdurEPSP*samplerate):round((halfdurEPSP+analyseWindow)*samplerate)].max()
HalfMaxWidth = [((np.abs(meanAllStim[i][meanAllStim[i][meanAllStim[i]==Max[i]].index[0]:] - Max[i]/2)).argmin() -round(halfdurEPSP*samplerate) + meanAllStim[i][meanAllStim[i]==Max[i]].index[0])/samplerate*1000 for i in meanAllStim.columns]

# Plot
plt.close()
fig, axs = plt.subplots(2,2, figsize=(12,6))
fig.suptitle(f'Mouse = {mouse}, LFP = {Selected_region} \nfilename = {folder_base.parts[-6]}', fontsize=12) # Change to indicate where is located that LFP

time_axis = np.linspace(-halfdurEPSP*1000, halfdurEPSP*1000, meanAllStim.shape[0]) 
grey_shades = np.linspace(0.9, 0.1, len(meanAllStim.columns)) 
for i, col in enumerate(meanAllStim.columns):
    axs[0,0].plot(time_axis, meanAllStim[col], color=(grey_shades[i], grey_shades[i], grey_shades[i]), label=f'{round(col,4)} ms')
axs[0,0].plot(time_axis, np.mean(meanAllStim, axis= 1), color='red')
axs[0,0].set(xlabel="Time (ms)", ylabel="Averaged EPSPs")
#axs[0,0].set(ylim=(-750, 1000))
axs[0,0].set(xlim=(-25, 50))

axs[0,1].plot(DurStim,Amplitude, 'k')
axs[0,1].set(xlabel="Opto stim duration (ms)", ylabel="Amplitude ")

axs[1,0].plot(DurStim,Max, 'k')
axs[1,0].set(xlabel="Opto stim duration (ms)", ylabel="Maximum ")

axs[1,1].plot(DurStim, HalfMaxWidth, 'k')
axs[1,1].set(xlabel="Opto stim duration (ms)", ylabel="HalfWidth (ms)")

plt.tight_layout()
axs[0,0].legend(loc='upper center', bbox_to_anchor=(2.5, 1.2),
          fancybox=True, shadow=True, ncol=max(int(nb_differentStim/18), 1))
plt.subplots_adjust(right=.8)  
plt.show()